In [ ]:
from utils import *
import tensorflow as tf
from keras import backend as K

Check if keras is using GPU

In [ ]:
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
K._get_available_gpus()

Get tickers

In [ ]:
tickers_sets_path = 'data/tickers/sets/'

sets_files = sorted([tickers_sets_path + x for x in listdir(tickers_sets_path) if 'json' in x], reverse=True)

tickers_sets = []

for set_file in sets_files:
    with open(set_file, 'r') as f:
        tickers_sets.append(json.load(f))

Read dataset

In [ ]:
df = pd.read_csv('data/dataset.csv')
df = df.set_index('Date')
df = df[:-60]

Define training sets and data size

In [ ]:
in_sets = [0]
out_sets = [0]
data_size = 4800

input_tickers = set()
output_tickers = set()
for in_set in in_sets:
    input_tickers = input_tickers.union(set(tickers_sets[in_set]))

for out_set in out_sets:
    output_tickers = output_tickers.union(set(tickers_sets[out_set]))

input_tickers = list(input_tickers)


Process data

In [ ]:
scaler, x, y, dic = dataa(df, data_size, input_tickers, output_tickers, \
                        step_size=1, input_size=60, output_size=20, \
                        feature_range=(0,1), return_log=False)

Split dataset i training, validation and test

In [ ]:
x_train, y_train, x_val, y_val, x_test, y_test = train_val_test_split(x,y)

Define hyperparameters

In [ ]:
input_shape = (x.shape[1],x.shape[2])
output_shape = (y.shape[1], y.shape[2])
layers_info_list = [
    [
        (450, 0.2, True), 
        (600, 0.2, True),
        (600, 0.2, True),
        (400, 0.2, False)
    ],
    [
        (1000, 0.15, True), 
        (1000, 0.15, True),
        (1000, 0.15, False)
    ]
]


Create Models

In [ ]:
models = []
hyperparameters = [0,1]

for i in hyperparameters:
    layers_info = layers_info_list[i]
    models.append(create_model(input_shape, output_shape, layers_info, metrics=['accuracy', 'mse']))

Train networks

In [ ]:
history_list = []
epochs_list = [
    150,
    150
]

set_name = 'set6/'
plot_path_dir = 'metrics/' + set_name
extra_info = ''

j=0
for i in hyperparameters:

    model = models[j]
    epochs = epochs_list[j]
    j += 1
    try:
        
        history = model.fit(
            x=x_train,
            y=y_train,
            epochs=epochs,
            batch_size=32,
            validation_data=(x_val, y_val)
        )

        history_list.append(history)

        model_name = 'LSTM' + str(i) + extra_info
        # model.save('models/' + set_name + model_name + '/model')

        plot_path_loss = plot_path_dir + 'LOSS_' + model_name + '.png'
        validation_plot(history.history['loss'], history.history['val_loss'], 'train vs validation loss', f=lambda: plt.savefig(plot_path_loss))

        plot_path_mse = plot_path_dir + 'MSE_' + model_name + '.png'
        validation_plot(history.history['mse'], history.history['val_mse'], 'train vs validation mse', f=lambda: plt.savefig(plot_path_mse), plot_type='mse')

        plot_path_accuracy = plot_path_dir + 'ACCURACY_' + model_name + '.png'
        validation_plot(history.history['accuracy'], history.history['val_accuracy'], 'train vs validation accuracy', f=lambda: plt.savefig(plot_path_accuracy), plot_type='accuracy')

    except:

        print('ERRO NO MODELO ' + str(i))
    

In [ ]:
for model in models:
    y_predict = model.predict(x_val)
    mse_ = np.square(np.subtract(y_val,y_predict)).mean(axis=1)
    var_ = np.var(np.subtract(y_val,y_predict), axis=1)
    e_ = np.sum(np.mean(np.sqrt(np.add(mse_,var_)),axis=0))
    

In [ ]:
scaler2, x2, y2, dic2 = dataa(df, data_size, input_tickers, output_tickers, \
                        step_size=1, input_size=60, output_size=20, \
                        feature_range=(0,1), return_log=True)

In [ ]:
x2_train, y2_train, x2_val, y2_val, x2_test, y2_test = train_val_test_split(x,y)

In [ ]:
models2 = []
hyperparameters = [0,1]

for i in hyperparameters:
    layers_info = layers_info_list[i]
    models2.append(create_model(input_shape, output_shape, layers_info, metrics=['accuracy', 'mse']))

In [ ]:
history2_list = []
epochs_list = [
    150,
    150
]

set_name = 'set6/'
plot_path_dir = 'metrics/' + set_name
extra_info = '_RETURN_LOG'

j=0
for i in hyperparameters:

    model = models2[j]
    epochs = epochs_list[j]
    j += 1
    try:
        
        history = model.fit(
            x=x2_train,
            y=y2_train,
            epochs=epochs,
            batch_size=32,
            validation_data=(x2_val, y2_val)
        )

        history2_list.append(history)

        model_name = 'LSTM' + str(i) + extra_info
        # model.save('models/' + set_name + model_name + '/model')

        plot_path_loss = plot_path_dir + 'LOSS_' + model_name + '.png'
        validation_plot(history.history['loss'], history.history['val_loss'], 'train vs validation loss', f=lambda: plt.savefig(plot_path_loss))

        plot_path_mse = plot_path_dir + 'MSE_' + model_name + '.png'
        validation_plot(history.history['mse'], history.history['val_mse'], 'train vs validation mse', f=lambda: plt.savefig(plot_path_mse), plot_type='mse')

        plot_path_accuracy = plot_path_dir + 'ACCURACY_' + model_name + '.png'
        validation_plot(history.history['accuracy'], history.history['val_accuracy'], 'train vs validation accuracy', f=lambda: plt.savefig(plot_path_accuracy), plot_type='accuracy')

    except:

        print('ERRO NO MODELO ' + str(i))
    